In [20]:
import pandas as pd

class NJCleaner:

    def __init__(self, csv_path:str):
        df = pd.read_csv(csv_path)
        self.data = df

    def order_by_scheduled_time(self) -> pd.DataFrame:
        self.data = self.data.sort_values(by=['scheduled_time'])
        return self.data
    
    def drop_columns_and_nan(self) -> pd.DataFrame:
        self.data = self.data.drop(['from', 'to'], axis=1)
        self.data = self.data.dropna()
        return self.data

    def convert_date_to_day(self) -> pd.DataFrame:
        self.data['date'] = pd.to_datetime(self.data['date'])
        self.data['day'] = self.data['date'].dt.day_name()
        self.data = self.data.drop(['date'], axis=1)
        return self.data
    
    def convert_scheduled_time_to_part_of_the_day(self) -> pd.DataFrame:
        df = pd.DatetimeIndex(self.data['scheduled_time']).hour.astype('float')
        self.data['part_of_the_day'] = pd.cut(df, bins=[0, 3, 7, 11, 15, 19, 23], 
                                              labels=[ 'late_night', 'early_morning', 'morning', 'afternoon', 'evening', 'night']) 
        self.data.drop(['scheduled_time'], axis=1, inplace=True) 
        return self.data
    
    def convert_delay(self) -> pd.DataFrame:
        max = self.data['delay_minutes'].astype('float').max()
        self.data['delay'] = pd.cut(self.data['delay_minutes'].astype('float'), bins=[-1, 5, max], labels=['0', '1'])
        return self.data
    
    def drop_unnecessary_columns(self) -> pd.DataFrame:
        self.data.drop(['train_id', 'actual_time', 'delay_minutes'], axis=1, inplace=True)
        return self.data
    
    def save_first_60k(self, path:str):
        cutted_df = self.data.head(60000)
        cutted_df.to_csv(path)

    def prep_df(self, path:str='data/NJ.csv'):
        self.order_by_scheduled_time()
        self.drop_columns_and_nan()
        self.convert_date_to_day()
        self.convert_scheduled_time_to_part_of_the_day()
        self.convert_delay()
        self.drop_unnecessary_columns()
        self.save_first_60k(path)

In [21]:
nj_cleaner = NJCleaner("2018_03.csv")
ordered_df = nj_cleaner.data
ordered_df = nj_cleaner.order_by_scheduled_time()
ordered_df

,date,train_id,stop_sequence,from,from_id,to,to_id,scheduled_time,actual_time,delay_minutes,status,line,type
5283,2018-03-01,3806,1.0,Trenton,148,Trenton,148,2018-03-01 03:48:00,2018-03-01 04:02:07,14.116667,departed,Northeast Corrdr,NJ Transit
1270,2018-03-01,0042,1.0,Port Jervis,123,Port Jervis,123,2018-03-01 03:50:00,2018-03-01 03:50:04,0.066667,departed,Bergen Co. Line,NJ Transit
5284,2018-03-01,3806,2.0,Trenton,148,Hamilton,32905,2018-03-01 03:54:00,2018-03-01 04:02:07,8.116667,departed,Northeast Corrdr,NJ Transit
208,2018-03-01,3202,1.0,Long Branch,74,Long Branch,74,2018-03-01 03:58:00,2018-03-01 03:58:01,0.016667,departed,No Jersey Coast,NJ Transit
5285,2018-03-01,3806,3.0,Hamilton,32905,Princeton Junction,125,2018-03-01 04:00:00,2018-03-01 04:02:07,2.116667,departed,Northeast Corrdr,NJ Transit
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256459,2018-03-31,A663,NaN,Trenton,148,Philadelphia,1,NaN,2018-03-31 11:20:08,NaN,departed,KEYSTONE,Amtrak
256460,2018-03-31,A664,NaN,Philadelphia,1,Philadelphia,1,NaN,2018-03-31 11:26:07,NaN,departed,Amtrak,Amtrak
256461,2018-03-31,A664,NaN,Philadelphia,1,Trenton,148,NaN,2018-03-31 12:07:04,NaN,departed,Amtrak,Amtrak
256462,2018-03-31,A664,NaN,Trenton,148,Newark Penn Station,107,NaN,2018-03-31 12:47:04,NaN,departed,Amtrak,Amtrak


In [ ]:
dropped_df = nj_cleaner.drop_columns_and_nan()
dropped_df

In [ ]:
day_df = nj_cleaner.convert_date_to_day()
day_df

In [ ]:
part_of_the_day_df = nj_cleaner.convert_scheduled_time_to_part_of_the_day()
part_of_the_day_df

In [ ]:
delay_df = nj_cleaner.convert_delay()
delay_df

In [ ]:
unnec_df = nj_cleaner.drop_unnecessary_columns()
unnec_df

In [ ]:
head_df = nj_cleaner.save_first_60k('data/NJ.csv')

In [ ]:
njcleaner = NJCleaner("2018_03.csv")
njcleaner.prep_df()